<a href="https://colab.research.google.com/github/Jammyeong/MachineLearningClass/blob/main/7thWeek/Tugas_7_ML_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.ensemble import BaggingRegressor, GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import LabelEncoder

In [2]:
# URL dataset dari CMU
data_url = "http://lib.stat.cmu.edu/datasets/boston"

# Membaca dataset
raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)

# Memisahkan fitur dan target
X = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])  # Fitur
y = raw_df.values[1::2, 2]  # Target

url = "https://raw.githubusercontent.com/selva86/datasets/master/BostonHousing.csv"
boston_columns = [
    'CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS',
    'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT', 'MEDV'
]

# Load the uploaded datasets
automobile_df = pd.read_csv('/content/drive/MyDrive/smt akhir/ml/Automobile.csv')
boston_df = pd.read_csv(url,  header=None, names=boston_columns)
infrared_df = pd.read_csv('/content/drive/MyDrive/smt akhir/ml/Infrared.csv')

# Display the first few rows of each dataset
print('AUTOMOBILE')
print(automobile_df.head)
print()
print('BOSTON HOUSE')
print(boston_df.head)
print()
print('INFRARED')
print(infrared_df.head)

AUTOMOBILE
<bound method NDFrame.head of        price  highway-mpg  city-mpg  peak-rpm  horsepower  compression-ratio  \
0    13495.0           27        21    5000.0       111.0                9.0   
1    16500.0           27        21    5000.0       111.0                9.0   
2    16500.0           26        19    5000.0       154.0                9.0   
3    13950.0           30        24    5500.0       102.0               10.0   
4    17450.0           22        18    5500.0       115.0                8.0   
..       ...          ...       ...       ...         ...                ...   
200  16845.0           28        23    5400.0       114.0                9.5   
201  19045.0           25        19    5300.0       160.0                8.7   
202  21485.0           23        18    5500.0       134.0                8.8   
203  22470.0           27        26    4800.0       106.0               23.0   
204  22625.0           25        19    5400.0       114.0                9.5   

In [7]:
def preprocess_dataset(df, target_column):
    df = df.copy()
    df = df.dropna(subset=[target_column])  # buang baris tanpa target
    df = df.fillna(df.median(numeric_only=True))  # isi NaN dengan median

    for col in df.select_dtypes(include='object').columns:
        df[col] = LabelEncoder().fit_transform(df[col])  # ubah kategori ke angka

    X = df.drop(target_column, axis=1)
    y = df[target_column]
    return train_test_split(X, y, test_size=0.2, random_state=42)

def train_and_evaluate(X_train, X_test, y_train, y_test, dataset_name):
    print(f"\n {dataset_name.upper()}")

    # Bagging
    bag_model = BaggingRegressor(random_state=42)
    bag_model.fit(X_train, y_train)
    y_pred_bag = bag_model.predict(X_test)

    # Boosting
    boost_model = GradientBoostingRegressor(random_state=42)
    boost_model.fit(X_train, y_train)
    y_pred_boost = boost_model.predict(X_test)

    # Evaluasi
    for name, y_pred in zip(["Bagging", "Boosting"], [y_pred_bag, y_pred_boost]):
        mse = mean_squared_error(y_test, y_pred)
        rmse = mse ** 0.5
        r2 = r2_score(y_test, y_pred)
        print(f"\nModel: {name}")
        print(f"MSE : {mse:.2f}")
        print(f"RMSE: {rmse:.2f}")
        print(f"R²  : {r2:.2f}")

In [8]:
# Automobile dataset
X_train_a, X_test_a, y_train_a, y_test_a = preprocess_dataset(automobile_df, 'price')
train_and_evaluate(X_train_a, X_test_a, y_train_a, y_test_a, "Automobile")

# Boston dataset
X_train_b, X_test_b, y_train_b, y_test_b = preprocess_dataset(boston_df, 'MEDV')
train_and_evaluate(X_train_b, X_test_b, y_train_b, y_test_b, "Boston Housing")

# Infrared dataset
X_train_i, X_test_i, y_train_i, y_test_i = preprocess_dataset(infrared_df, 'aveOralM')
train_and_evaluate(X_train_i, X_test_i, y_train_i, y_test_i, "Infrared")



 AUTOMOBILE

Model: Bagging
MSE : 10053989.04
RMSE: 3170.80
R²  : 0.92

Model: Boosting
MSE : 5968222.37
RMSE: 2442.99
R²  : 0.95

 BOSTON HOUSING

Model: Bagging
MSE : 2359.86
RMSE: 48.58
R²  : 0.33

Model: Boosting
MSE : 2145.66
RMSE: 46.32
R²  : 0.39

 INFRARED

Model: Bagging
MSE : 0.07
RMSE: 0.26
R²  : 0.69

Model: Boosting
MSE : 0.05
RMSE: 0.22
R²  : 0.77


### **Penjelasan Matematis Metrik Evaluasi Regresi**

Berikut adalah rincian matematis dari metrik yang umum digunakan untuk mengevaluasi kualitas model regresi.

---

### 1. Mean Squared Error (MSE)

MSE mengukur rata-rata dari kuadrat selisih (error) antara nilai aktual dan nilai prediksi. Metrik ini memberikan 'beban' yang lebih berat pada error yang besar karena proses kuadrat.

**Rumus Matematis:**

$$
MSE = \frac{1}{n} \sum_{i=1}^{n} (y_i - \hat{y}_i)^2
$$

**Penjelasan Variabel:**
* `$n$`: Jumlah total titik data.
* `$y_i$`: Nilai aktual (sebenarnya) untuk data ke-i.
* `$\hat{y}_i$`: Nilai yang diprediksi oleh model untuk data ke-i.

**Interpretasi:** **Semakin rendah nilainya (mendekati 0), semakin baik.** Nilai yang lebih kecil menunjukkan model memiliki rata-rata error kuadrat yang lebih sedikit.

---

### 2. Root Mean Squared Error (RMSE)

RMSE adalah akar kuadrat dari MSE. Keuntungannya adalah satuannya sama dengan variabel target, sehingga lebih mudah diinterpretasikan sebagai 'rata-rata besar error' dari model.

**Rumus Matematis:**

$$
RMSE = \sqrt{MSE} = \sqrt{\frac{1}{n} \sum_{i=1}^{n} (y_i - \hat{y}_i)^2}
$$

**Penjelasan Variabel:**
* Variabelnya sama seperti pada MSE.

**Interpretasi:** **Semakin rendah nilainya (mendekati 0), semakin baik.** Misalnya, jika Anda memprediksi harga rumah dalam Rupiah, nilai RMSE sebesar 50.000.000 berarti prediksi Anda secara rata-rata meleset sekitar Rp 50 juta.

---

### 3. R-Squared (R² atau Koefisien Determinasi)

R-Squared mengukur seberapa besar proporsi (persentase) varians dalam variabel target yang dapat dijelaskan oleh model. Ini menunjukkan seberapa baik model 'cocok' dengan data dibandingkan dengan model sederhana yang hanya memprediksi nilai rata-rata.

**Rumus Matematis:**

$$
R^2 = 1 - \frac{SS_{res}}{SS_{tot}} = 1 - \frac{\sum_{i=1}^{n} (y_i - \hat{y}_i)^2}{\sum_{i=1}^{n} (y_i - \bar{y})^2}
$$

**Penjelasan Variabel:**
* `$SS_{res}$`: *Sum of Squares of Residuals*, yaitu jumlah kuadrat dari error prediksi.
* `$SS_{tot}$`: *Total Sum of Squares*, yaitu jumlah kuadrat selisih antara setiap nilai aktual dan nilai rata-ratanya.
* `$y_i$`: Nilai aktual.
* `$\hat{y}_i$`: Nilai prediksi.
* `$\bar{y}$`: Nilai rata-rata (mean) dari semua nilai aktual `$y$`.

**Interpretasi:** Skor berkisar antara 0 hingga 1. **Semakin tinggi nilainya (mendekati 1), semakin baik.** Nilai R² sebesar 0.85 berarti model Anda dapat menjelaskan 85% dari variasi yang ada pada data target.